In [1]:
import numpy as np
import csv
import time

np.random.seed(42)
def randomize(): np.random.seed(time.time())

In [ ]:
RND_MEAN = 0.0
RND_STD = 0.003
LEARNING_RATE = 0.001

In [2]:
def steel_exce(epochs=10, batch_size=10, report=1):
    load_steel_dataset()
    init_model()
    train_and_test(epochs, batch_size, report)

In [3]:
def load_steel_dataset():
    with open('dataset/faults.csv', 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
        
    global data, input_dim, output_dim
    input_dim, output_dim = 27, 7
    data = np.asarray(rows, dthpe=np.float32)

In [15]:
def init_model():
    global weights, bias, input_dim, output_dim
    weights = np.random.normal(RND_MEAN, RND_STD, [input_dim, output_dim])
    bias = np.zeros([output_dim])

In [ ]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

In [ ]:
def run_train(x, y):

    #wx + b 계산
    output, aux_nn = forward_neuralnet(x)
    #(wx + b - y) ^ 2 계산
    loss, aux_pp = forward_postproc(output, y)
    # wx + b 와 y 간의
    accuracy = eval_accuracy(output, y)
    
    #dL/dL
    dL_dL = 1.0
    G_output = backprop_postproc(dL_dL, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [ ]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_dim
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_dim], test_data[:, -output_dim:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_dim
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_dim], train_data[:, -output_dim:]

In [ ]:
def forward_neuralnet(x):
    global weights, bias
    # x : 1 x 10
    # weights : 10 x 1
    # bias : 1 x 1
    output = np.matmul(x, weights) + bias
    return output, x

def backprop_neuralnet(G_upstream, x):
    global weights, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_upstream)
    G_b = np.sum(G_upstream, axis=0)

    weights -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [4]:
def forward_postproc(output, y):
    #output : logit
    #y : 
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux

    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy

    return G_output

In [5]:
def eval_accuracy(output, y):
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)

    return np.mean(correct)

In [14]:
def softmax(x):
    #x : N x 27

    #max_elem : N,
    max_elem = np.max(x, axis=1)
    #diff : N x 27
    diff = (x.T - max_elem).T
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.T / sum_exp).T
    return probs

def softmax_derv(x, y):
    batch_size, element_size = x.shape
    derv = np.ndarray([batch_size, element_size, element_size])
    for n in range(batch_size):
        for i in range(element_size):
            for j in range(element_size):
                #dy_j/dx_i 
                derv[n, i , j] = -y[n, i] * y[n, j]
            derv[n, i, i] += y[n, i]

    return derv

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs + 1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

(27, 10)